In [ ]:
from datasets import load_dataset
import json, os, random
import dotenv
from huggingface_hub import login
# OPTIMIZED VERSION - Process subset for faster development
from tqdm import tqdm
import warnings

dotenv.load_dotenv()

HF_TOKEN = os.getenv("HUGGINGFACE_ACCESS_TOKEN")
login(token=HF_TOKEN)

OUTDIR = "direct_response"
os.makedirs(OUTDIR, exist_ok=True)
train_path = f"{OUTDIR}/train.jsonl"
val_path   = f"{OUTDIR}/val.jsonl"


/opt/anaconda3/envs/cs224n-cpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Suppress the metadata warning (this is harmless - just missing README metadata)
warnings.filterwarnings("ignore", message="Repo card metadata block was not found")

dataset = load_dataset("socratesft/SocSci210", token=HF_TOKEN)
# Verify the dataset loaded successfully
print(f"✓ Dataset loaded: {len(dataset['train'])} examples")
print(f"✓ Columns: {list(dataset['train'].features.keys())}") 

Repo card metadata block was not found. Setting CardData to empty.


✓ Dataset loaded: 2901390 examples
✓ Columns: ['sample_id', 'participant', 'demographic', 'stimuli', 'response', 'condition_num', 'task_num', 'prompt', 'reasoning', 'study_id']


In [12]:
dataset = dataset["train"]

In [15]:
def filter_by_demographics(dataset, demographic_filters):
    """    
    Returns:
        Filtered dataset containing only examples that match ALL specified criteria
        and have ALL required demographic keys present
    
    Examples:
        # Filter for females only
        filtered = filter_by_demographics(ds, {"gender": "Female"})
        
        # Filter for married females with college education
        filtered = filter_by_demographics(ds, {
            "gender": "Female", 
            "marital_status": "Married",
            "education": "Post grad study/professional degree"
        })
    """
    def matches_criteria(example):
        # Check if demographic field exists
        if 'demographic' not in example or example['demographic'] is None:
            return False
            
        demographic = example['demographic']
        
        # Check if all required keys exist in demographic data
        for key in demographic_filters.keys():
            if key not in demographic:
                return False
        
        # Check if all criteria match
        for key, required_value in demographic_filters.items():
            actual_value = demographic.get(key)
            if actual_value != required_value:
                return False
                
        return True
    
    # Filter the dataset
    if hasattr(dataset, 'filter'):  # HuggingFace dataset
        return dataset.filter(matches_criteria)
    else:  # Regular list/iterable
        return [ex for ex in dataset if matches_criteria(ex)]

# Test the function with a sample
if 'dataset' in globals():
    print(f"\n📊 Dataset info:")
    print(f"Total examples: {len(dataset):,}")
    
    # Test filter for females
    try:
        filtered_dataset = filter_by_demographics(dataset, {"gender": "Male"})
    
        print(f"Filtered dataset size: {len(filtered_dataset):,}")
        
    except Exception as e:
        print(f"Error testing filter: {e}")
else:
    print("\ndataset' not loaded yet. Run the previous cells first!")


📊 Dataset info:
Total examples: 2,901,390
Filtered dataset size: 791,278


In [16]:

# USE FILTERED DATASET FROM ABOVE HERE
ds = filtered_dataset
print(f"Full dataset size: {len(ds):,} examples")


# Optional single system instruction
SYSTEM_TXT =  "You are simulating a survey respondent. Answer exactly as instructed, following the specified response format without additional commentary."

subset = ds

rows = []
processed = 0
skipped = 0

for ex in tqdm(subset, desc="Converting to chat format"):
    prompt = ex.get("prompt") .strip()
    response = ex.get("response")
    
    if not prompt or response is None:
        skipped += 1
        continue

    assistant = str(response).strip()
    rows.append({
        "messages": [
            {"role": "system", "content": SYSTEM_TXT},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": assistant}
        ]
    })
    processed += 1

print(f"✓ Processed: {processed:,} examples, Skipped: {skipped:,} examples")

# Simple split
random.seed(42)
random.shuffle(rows)
k = max(100, int(0.02 * len(rows)))  # ~2% for validation
val, train = rows[:k], rows[k:]

print(f"Writing files: {len(train):,} train, {len(val):,} validation...")

# Write files (FIXED - no more slicing errors!)
with open(train_path, "w") as f:
    for r in train: 
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(val_path, "w") as f:
    for r in val:   
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✓ Successfully wrote:")
print(f"  - {train_path}: {len(train):,} examples") 
print(f"  - {val_path}: {len(val):,} examples")


Full dataset size: 791,278 examples


Converting to chat format: 100%|██████████| 791278/791278 [01:02<00:00, 12699.63it/s]


✓ Processed: 791,278 examples, Skipped: 0 examples
Writing files: 775,453 train, 15,825 validation...
✓ Successfully wrote:
  - direct_response/train.jsonl: 775,453 examples
  - direct_response/val.jsonl: 15,825 examples
